In [5]:
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("You are a helpful assistant that defines words. Define this word: {word}")
llm=GPT4All(
 model="./falcon.bin"

)
chain = prompt | llm
chain.invoke({
  "word":"potato",
})

'\nA potato is a starchy, tuberous crop from the perennial plant Solanum tuberosum. It is one of the most widely cultivated and consumed vegetables in the world. Potatoes are often boiled, baked, fried, or mashed as a side dish or ingredient in various dishes.'

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# def get_weather(lon, lat):
#   return "hahaha"

function = {
  "name": "create_quiz", 
  "description":"function that takes list of questions and answers and returns a quiz",
  "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },

        "required":["questions"],
},
}
llm = ChatOpenAI(temperature=0.1).bind(
  function_call = {
    "name": "create_quiz"
  },
  functions=[
    function
  ]
)
prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({
  "city":"rome"
})
response = response.additional_kwargs["function_call"]["arguments"]
import json
json.loads(response)

{'questions': [{'question': 'What year was Rome founded?',
   'answers': [{'answer': '753 BC', 'correct': True},
    {'answer': '476 AD', 'correct': False},
    {'answer': '1000 BC', 'correct': False}]},
  {'question': 'Who was the first emperor of Rome?',
   'answers': [{'answer': 'Julius Caesar', 'correct': False},
    {'answer': 'Augustus', 'correct': True},
    {'answer': 'Nero', 'correct': False}]},
  {'question': 'What famous structure in Rome was built by the ancient Romans and is still standing today?',
   'answers': [{'answer': 'Colosseum', 'correct': True},
    {'answer': 'Pantheon', 'correct': False},
    {'answer': 'Trevi Fountain', 'correct': False}]}]}

In [ ]:
for question in json.loads(response)['questions']:
  print(question)

'?\nA: A starchy tuberous vegetable that is a staple food in many cultures, grown in rows in the ground and harvested when the plants die back. It is a popular food source and is used in a variety of dishes, including soups, salads, and fries.'

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

def invoke_chain(question):
    result=chain.invoke(question)
    memory.save_context({"input":question}, {"output":result.content})
    return result


memory = ConversationBufferMemory(return_return_messages= True)

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chap_1.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)


chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm 
)
ans1 = invoke_chain("Is Aaronson guilty?")
ans2 = invoke_chain("What message did he write in the table?")
ans3 = invoke_chain("Who is Julia?")
print(f"{ans1}\n {ans2} \n {ans3}")
memory.load_memory_variables({})

content='Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'
 content='He wrote "2+2=5" in the dust on the table.' 
 content='Julia is a character mentioned in the text. She is someone who Winston loves and cares for deeply, and he has a strong emotional connection to her.'


{'history': 'Human: Is Aaronson guilty?\nAI: Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.\nHuman: What message did he write in the table?\nAI: He wrote "2+2=5" in the dust on the table.\nHuman: Who is Julia?\nAI: Julia is a character mentioned in the text. She is someone who Winston loves and cares for deeply, and he has a strong emotional connection to her.'}

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

def invoke_chain(question):
    result=chain.invoke(question)
    memory.save_context({"input":question}, {"output":result.content})
    return result


memory = ConversationBufferMemory(return_return_messages= True)

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chap_1.txt")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)


chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm 
)
ans1 = invoke_chain("Is Aaronson guilty?")
ans2 = invoke_chain("What message did he write in the table?")
ans3 = invoke_chain("Who is Julia?")
print(f"{ans1}\n {ans2} \n {ans3}")
memory.load_memory_variables({})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

def invoke_chain(question):
    result=chain.invoke(question)
    memory.save_context({"input":question}, {"output":result.content})
    return result

def save_message( question):
    print(question)
    # print(content)
    #  memory.save_context({"input":question}, {"output":content})

memory = ConversationBufferMemory(return_return_messages= True)

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chap_1.txt")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)


chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    
)
result = chain.invoke("Is Aaronson guilty?")

result

# ans1 = invoke_chain("Is Aaronson guilty?")
# ans2 = invoke_chain("What message did he write in the table?")
# ans3 = invoke_chain("Who is Julia?")
# print(f"{ans1}\n {ans2} \n {ans3}")
# memory.load_memory_variables({})

In [ ]:
ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant. ......)]'), HumanMessage(content='Is Aaronson guilty?')])

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

def plus(a,b):
  return a+b
llm=ChatOpenAI(temperature=0.1)
agent = initialize_agent(
  llm=llm, 
  verbose=True,
  agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
  tools=[
  StructuredTool.from_function(
    func=plus, 
    name="Sum calculator", 
    description="Use this to perform sums of two numbers. This tool take two arguments, both should be numbers",)
])

prompt="Cost of $355.39 + $924.87 + $721,2 + $1940.29 + $573.63 + $65.72 + $35 + $552.00 + $76.16 + $29.12"
agent.invoke(prompt)


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

def plus(inputs):
  a,b = inputs.split(",")
  return float(a)+float(b)
llm=ChatOpenAI(temperature=0.1)
agent = initialize_agent(
  llm=llm, 
  verbose=True,
  agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
  handle_parsing_errors=True,
  tools=[
  Tool.from_function(
    func=plus, 
    name="Sum calculator", 
    description="Use this to perform sums of two numbers. Use this  tool by sending a pair of number separated by comma. \n Example:1,2",)
])

prompt="Cost of $355.39 + $924.87 + $721,2 + $1940.29 + $573.63 + $65.72 + $35 + $552.00 + $76.16 + $29.12"
agent.invoke(prompt)




> Entering new AgentExecutor chain...
I need to add up all the given numbers to find the total cost.
Action: Sum calculator
Action Input: 355.39,924.87
Observation: 1280.26
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum calculator
Action Input: 1280.26,721.2
Observation: 2001.46
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum calculator
Action Input: 2001.46,1940.29
Observation: 3941.75
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum calculator
Action Input: 3941.75,573.63
Observation: 4515.38
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum calculator
Action Input: 4515.38,65.72
Observation: 4581.1
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum calculator
Action Input: 4581.1,35
Observation: 4616.1
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum calculator
Action Input: 4616.1,552.

{'input': 'Cost of $355.39 + $924.87 + $721,2 + $1940.29 + $573.63 + $65.72 + $35 + $552.00 + $76.16 + $29.12',
 'output': '$5273.38'}

In [14]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel , PositiveInt, Field



class CalculatorToolArgsSchema(BaseModel):
  a:float = Field(description="The first number")
  b:float = Field(description="The second number")

class CalculatorTool(BaseModel):
  name="CalculatorTool"
  description="""
  Use this to perform sums of two numbers.
  The first and second arguments should be numbers.
  Only receives two arguments.
  """
  args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

  def _run(self, a,b):
    return a+b

llm=ChatOpenAI(temperature=0.1)
agent = initialize_agent(
  llm=llm, 
  verbose=True,
  agent=AgentType.OPENAI_FUNCTIONS,
  tools=[
    CalculatorTool()
])

prompt="Cost of $355.39 + $924.87 + $721,2 + $1940.29 + $573.63 + $65.72 + $35 + $552.00 + $76.16 + $29.12"
agent.invoke(prompt)

ValidationError: 1 validation error for OpenAIFunctionsAgent
tools -> 0
  Can't instantiate abstract class BaseTool with abstract method _run (type=type_error)

In [1]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm= ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float= Field(description="The first number")
    b: float= Field(description="The second number")


class CalculatorTool(BaseTool):
    name= "CalculatorTool"
    description= """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema]= CalculatorToolArgsSchema

    def _run(self, a, b):
      return a+ b


agent= initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt= "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [19]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm= ChatOpenAI(temperature=0.1)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
  query:str=Field(description="The query you will search for")
class StockMarketSymbolSearchTool(BaseTool):
  name="StockMarketSymbolSearchTool"
  description="""
  Use this tool to find the stock market symbol for a company.
  It takes a query as an argument.
  Example query: Stock market Symbol for Apple Company
  """
  args_schema:Type[StockMarketSymbolSearchToolArgsSchema]=StockMarketSymbolSearchToolArgsSchema

  def _run(self, query):
    ddg=DuckDuckGoSearchAPIWrapper()
    return ddg.run(query)



agent= initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt= "Give me information on CloudFlare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'CloudFlare stock'}`




HTTPError: 

In [5]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from langchain.tools import DuckDuckGoSearchResults
llm = ChatOpenAI(temperature=0.1)


class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Research about the xz backdoor"
    )


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = """
    Use this tool to find avout Research about the xz backdoor.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        WikipediaSearchToolArgsSchema
    ] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)
    # def _run(self, query):
    #     ddg = DuckDuckGoSearchResults()
    #     return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
    ],
)

prompt = "Research about the xz backdoor"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'xz backdoor'}`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been assigned a CVSS score of 10.0, the highest possible score.

Page: XZ Utils
Summary: XZ Utils (previously LZMA Utils) is a set of free software command-line lossless da

{'input': 'Research about the xz backdoor',
 'output': 'I found information about the xz backdoor. On 29 March 2024, a backdoor was discovered in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1. The backdoor targeted Debian- and RPM-based systems running on the x86-64 architecture. It allowed an attacker with a specific Ed448 private key to execute remote code on the affected Linux system. The issue was assigned a CVSS score of 10.0, the highest possible score.\n\nXZ Utils is a set of free software command-line lossless data compressors that includes programs like lzma and xz. It uses the Lempel–Ziv–Markov chain algorithm (LZMA) for compression/decompression.\n\nA backdoor is a covert method of bypassing normal authentication or encryption in a computer system. It can be used for securing remote access or obtaining access to plaintext in cryptosystems. Backdoors can take various forms, such as hidden parts of a program, separate programs, firmware code, or p